In [1]:
from pysmt.shortcuts import *  # Import all shortcuts from PySMT for creating logical expressions.
from pysmt.typing import BVType  # Import the bit-vector type for defining variables.

# Define the bit-vector size for our model.
n = 10  # This specifies that we will work with 10-bit vectors.

# Function to declare state variables for each state index i
def declare(i):
    state = {}
    state['pc'] = Symbol('pc' + str(i), BVType(n))
    state['x'] = Symbol('x' + str(i), BVType(n))
    state['y'] = Symbol('y' + str(i), BVType(n))
    state['a'] = Symbol('a' + str(i), BVType(n))
    state['b'] = Symbol('b' + str(i), BVType(n))
    state['z'] = Symbol('z' + str(i), BVType(n))
    return state

def init(state, x_val, y_val):
    half_max = 2 ** (n // 2)
    PC = Equals(state['pc'], BV(0, n))
    X = Equals(state['x'], BV(x_val, n))
    Y = Equals(state['y'], BV(y_val, n))
    A = Equals(state['a'], BV(x_val, n))  # Keep a equal to x_val
    B = Equals(state['b'], BV(y_val, n))  # Keep b equal to y_val
    Z = Equals(state['z'], BV(0, n))  # z starts at 0

    # Add constraints to ensure a < 2^(n/2) and b < 2^(n/2)
    constraint_a = BVULT(state['a'], BV(half_max, n))
    constraint_b = BVULT(state['b'], BV(half_max, n))

    # Combine all initial conditions including new constraints
    return And(PC, X, Y, A, B, Z, constraint_a, constraint_b)

# Transition conditions to simulate multiplication process
def trans(curr, prox):
    t01 = And(
        Equals(curr['pc'], BV(0, n)),
        Equals(prox['pc'], BV(1, n)),
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['a'], curr['a']),
        Equals(prox['b'], curr['b']),
        Equals(prox['z'], BV(0, n))  # z remains 0 for the first transition
    )

    t12 = And(
        Equals(curr['pc'], BV(1, n)),
        NotEquals(curr['y'], BV(0, n)),
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], BVSub(curr['y'], BV(1, n))),
        Equals(prox['a'], curr['a']),
        Equals(prox['b'], curr['b']),
        Equals(prox['z'], BVAdd(curr['z'], curr['x'])),  # Update z correctly here
        Equals(prox['pc'], BV(1, n))
    )

    t23 = And(
        Equals(curr['pc'], BV(1, n)),
        Equals(curr['y'], BV(0, n)),
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['a'], curr['a']),
        Equals(prox['b'], curr['b']),
        Equals(prox['z'], curr['z']),  # z remains unchanged here
        Equals(prox['pc'], BV(2, n))
    )

    t33 = And(
        Equals(curr['pc'], BV(2, n)),
        Equals(prox['pc'], BV(2, n)),
        Equals(prox['x'], curr['x']),
        Equals(prox['y'], curr['y']),
        Equals(prox['a'], curr['a']),
        Equals(prox['b'], curr['b']),
        Equals(prox['z'], curr['z'])  # z remains unchanged
    )

    return Or(t01, t12, t23, t33)

# Invariant that x * y + z = a * b
def invariant_check(state):
    return Equals(
        BVAdd(BVMul(state['x'], state['y']), state['z']),
        BVMul(state['a'], state['b'])
    )

# Overflow condition
def overflow(state):
    max_val = BV((2**n) - 1, n)
    return Or(
        BVUGT(state['x'], max_val),
        BVUGT(state['y'], max_val),
        BVUGT(state['a'], max_val),
        BVUGT(state['b'], max_val),
        BVUGT(state['z'], max_val)
    )

# Function to print state values in the desired format
def print_state(k, state, solver):
    print(f"Estado {k}: pc = {solver.get_value(state['pc']).bv_unsigned_value()}, "
          f"x = {solver.get_value(state['x']).bv_unsigned_value()}, "
          f"y = {solver.get_value(state['y']).bv_unsigned_value()}, "
          f"a = {solver.get_value(state['a']).bv_unsigned_value()}, "
          f"b = {solver.get_value(state['b']).bv_unsigned_value()}, "
          f"z = {solver.get_value(state['z']).bv_unsigned_value()}")

# Bounded Model Checker function with detailed output
def bmc_always(inv, K, x_val, y_val):
    with Solver() as solver:
        states = [declare(i) for i in range(K + 1)]
        solver.add_assertion(init(states[0], x_val, y_val))

        for k in range(K):
            if k > 0:
                solver.add_assertion(trans(states[k - 1], states[k]))

            # Print the state values at each step
            solver.push()
            if solver.solve():
                print_state(k, states[k], solver)
            solver.pop()

            solver.push()
            # Check invariant and overflow separately for clearer diagnosis
            if solver.solve([Not(inv(states[k]))]):
                print(f"> Invariante falha no estado {k}")
                print_state(k, states[k], solver)
                solver.pop()
                return

            if solver.solve([overflow(states[k])]):
                print(f"> Overflow detetado no estado {k}")
                print_state(k, states[k], solver)
                solver.pop()
                return

            solver.pop()

        # Success message if all states verified
        print(f"> Invariante verifica-se para os primeiros {K} estados.")

        # Print final values of x * y + z and a * b for verification
        final_x = solver.get_value(states[K-1]['x']).bv_unsigned_value()
        final_y = solver.get_value(states[K-1]['y']).bv_unsigned_value()
        final_z = solver.get_value(states[K-1]['z']).bv_unsigned_value()
        final_a = solver.get_value(states[K-1]['a']).bv_unsigned_value()
        final_b = solver.get_value(states[K-1]['b']).bv_unsigned_value()
        
        expected_sum = final_x * final_y + final_z
        expected_product = final_a * final_b
        
        print(f"Resultado final: x * y + z = {expected_sum}, a * b = {expected_product}. Verificação: {'Correto' if expected_sum == expected_product else 'Incorreto'}.")

# Example test with random values
# Exemplo de teste com valores aleatórios para a e b
import random
for _ in range(3):
    a_val = random.randint(1, 2**(n // 2) - 1)  # Gerar valor aleatório para a
    b_val = random.randint(1, 2**(n // 2) - 1)  # Gerar valor aleatório para b
    print(f"Testando com a = {a_val}, b = {b_val}")
    bmc_always(invariant_check, 20, a_val, b_val)  # Passar a e b em vez de x e y


Testando com a = 26, b = 5
Estado 0: pc = 0, x = 26, y = 5, a = 26, b = 5, z = 0
Estado 1: pc = 1, x = 26, y = 5, a = 26, b = 5, z = 0
Estado 2: pc = 1, x = 26, y = 4, a = 26, b = 5, z = 26
Estado 3: pc = 1, x = 26, y = 3, a = 26, b = 5, z = 52
Estado 4: pc = 1, x = 26, y = 2, a = 26, b = 5, z = 78
Estado 5: pc = 1, x = 26, y = 1, a = 26, b = 5, z = 104
Estado 6: pc = 1, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 7: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 8: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 9: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 10: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 11: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 12: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 13: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 14: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 15: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 16: pc = 2, x = 26, y = 0, a = 26, b = 5, z = 130
Estado 17: pc = 2, x 

# Restrictions for Solvers in Bounded Model Checking

When utilizing solvers like Z3 or PySMT for bounded model checking of the integer multiplication automaton, it’s important to adhere to the following restrictions:

1. **Bit-Vector Size**:
   - The bit-vector size (`n`) must be explicitly defined. In the provided code, `n` is set to 10, meaning all bit-vector variables will represent integers within the range \([0, 2^{10}-1]\).
   - Ensure all operations respect the limits imposed by the bit-vector size to avoid overflow and undefined behavior.

2. **Initial Values**:
   - The initial values for `a` and `b` should be constrained to less than \(2^{(n/2)}\). This is to ensure that during the multiplication process, the resultant value will not exceed the maximum representable value in `n` bits.
   - For example, if `n = 10`, both `a` and `b` should be less than \(32\).

3. **Invariant Conditions**:
   - The invariant \(x \times y + z = a \times b\) must be maintained throughout the state transitions. If this invariant does not hold at any state, it indicates a logical error in the multiplication implementation.
   - The model checker must regularly assert that this invariant holds true after each transition between states.

4. **Overflow Checks**:
   - The solver must check for potential overflow conditions for all state variables. If any variable exceeds \(2^n - 1\), it indicates an overflow that the program must handle.
   - Each state variable (`x`, `y`, `a`, `b`, `z`) should be validated against the maximum value allowed for their respective bit-vectors.

5. **Transition Validity**:
   - All state transitions must be defined correctly and should maintain the logical structure of the multiplication algorithm.
   - The transitions should ensure that the state machine can only move to valid next states according to the defined logic of multiplication.

6. **Bounded Model Checking Depth (K)**:
   - The depth \(K\) defines how many states the model checker will verify. A larger \(K\) can provide more extensive verification but at the cost of increased computational complexity.
   - Be mindful of the trade-off between the thoroughness of the checks and the performance of the solver.

7. **Solver Limitations**:
   - Different solvers may have limitations in terms of the size of the formulas they can handle. Always check the documentation for the specific limitations of the solver you are using.
   - Performance can vary based on the structure of the constraints, so consider optimizing your constraints for better solver performance.

Adhering to these restrictions is essential for ensuring the correctness and reliability of the model checking process when verifying the automaton that performs integer multiplication using bit-vectors.
